In [1]:
Pkg.checkout("DistributedArrays","anj/map_localparts")

INFO: Checking out DistributedArrays anj/map_localparts...
INFO: Pulling DistributedArrays latest anj/map_localparts...


LoadError: fixed packages introduce conflicting requirements for KernelDensityEstimate: 
         Caesar requires versions [0.2.4,∞) [none of the available versions can satisfy this requirement]
       available versions are 0.0.1, 0.0.2, 0.0.3, 0.0.4, 0.1.0, 0.2.0, 0.2.1, 0.2.2 and 0.2.3

In [2]:
nprocs() == 1 && addprocs(4)
using DistributedArrays

INFO: Recompiling stale cache file /Users/edelman/.julia/lib/v0.5/DistributedArrays.ji for module DistributedArrays.


In [3]:
@everywhere function prefix!(op, v, v0 = 0)
    v[1] += v0
    for i = 2:length(v)
        v[i] = op(v[i-1], v[i])
    end
    return v
end

@everywhere function prefixlag!(op, v, v0 = 0)
    vi = v[1]
    v[1] = v0
    for i = 2:length(v)
        tmp  = op(v[i - 1], vi)
        vi   = v[i]
        v[i] = tmp
    end
    v
end

In [4]:
prefixlag!(+,collect(1:8))

8-element Array{Int64,1}:
  0
  1
  3
  6
 10
 15
 21
 28

In [5]:
A = prefix!(*, [[0 1; 1 1] for i = 1:10])

10-element Array{Array{Int64,2},1}:
 [0 1; 1 1]    
 [1 1; 1 2]    
 [1 2; 2 3]    
 [2 3; 3 5]    
 [3 5; 5 8]    
 [5 8; 8 13]   
 [8 13; 13 21] 
 [13 21; 21 34]
 [21 34; 34 55]
 [34 55; 55 89]

In [6]:
prefix!($, bitrand(10))

10-element BitArray{1}:
 false
 false
 false
  true
  true
 false
 false
 false
 false
 false

In [7]:
x1, x2, x3, x4 = 0:9, 10:19, 20:29, 30:39

(0:9,10:19,20:29,30:39)

In [8]:
x = Vector{Int}[x1, x2, x3, x4]

# Simulating local parts of 0:39 on four processors

4-element Array{Array{Int64,1},1}:
 [0,1,2,3,4,5,6,7,8,9]          
 [10,11,12,13,14,15,16,17,18,19]
 [20,21,22,23,24,25,26,27,28,29]
 [30,31,32,33,34,35,36,37,38,39]

In [9]:
y1 = sum.(x)


4-element Array{Int64,1}:
  45
 145
 245
 345

In [10]:
y2 = prefixlag!(+, y1)

4-element Array{Int64,1}:
   0
  45
 190
 435

In [11]:
y3 = map((t,s) -> prefix!(+, t, s), x, y2)

4-element Array{Array{Int64,1},1}:
 [0,1,3,6,10,15,21,28,36,45]              
 [55,66,78,91,105,120,136,153,171,190]    
 [210,231,253,276,300,325,351,378,406,435]
 [465,496,528,561,595,630,666,703,741,780]

In [12]:
cumsum(0:39)'

1×40 Array{Int64,2}:
 0  1  3  6  10  15  21  28  36  45  55  …  561  595  630  666  703  741  780

## Distributed version (requires some changes to DArrays)
Notice similarity to and differences from the sequential version

In [13]:
xd = @DArray [i for i = 0:39]

40-element DistributedArrays.DArray{Int64,1,Array{Int64,1}}:
  0
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
  ⋮
 28
 29
 30
 31
 32
 33
 34
 35
 36
 37
 38
 39

In [14]:
[DistributedArrays.chunk(xd, i) for i = 1:4]

4-element Array{Array{Int64,1},1}:
 [0,1,2,3,4,5,6,7,8,9]          
 [10,11,12,13,14,15,16,17,18,19]
 [20,21,22,23,24,25,26,27,28,29]
 [30,31,32,33,34,35,36,37,38,39]

In [16]:
yd1 = DistributedArrays.map_localparts(sum, xd)
#yd1 = map_localparts(sum,xd) 


RemoteException: On worker 2:
BoundsError
 in getindex at ./number.jl:21
 in getlocalindex at /Users/edelman/.julia/v0.5/DistributedArrays/src/core.jl:544
 in #625 at ./multi.jl:1421
 in run_work_thunk at ./multi.jl:1001
 in macro expansion at ./multi.jl:1421 [inlined]
 in #624 at ./event.jl:68

In [32]:
yd2 = prefixlag!(+, Array(yd1))

4-element Array{Int64,1}:
   0
  45
 190
 435

In [27]:
# Notice that we'll have to index into the offset of the prefix
# because `map_localpart` interprets each element as an Array
yd3 = map_localparts((t,s) -> prefix!(+, t, s[1]), xd, distribute(yd2))


40-element DistributedArrays.DArray{Int64,1,Array{Int64,1}}:
     0
     1
     4
    10
    20
    35
    56
    84
   120
   165
   220
   286
   364
   455
   560
   680
   816
   969
  1140
  1330
  1540
  1771
  2024
  2300
  2600
  2925
  3276
  3654
  4060
  4495
  4960
  5456
  5984
  6545
  7140
  7770
  8436
  9139
  9880
 10660

In [23]:
sum(vcat(x...) - yd3)

0

In [17]:
methods(map_localparts)

LoadError: UndefVarError: map_localparts not defined

In [45]:
sin(cos(tan(5)))

-0.8257855101654086

In [46]:
tan(5) |> cos |> sin

-0.8257855101654086

In [ ]:
&2, &1

In [20]:
true $ true

false